In [1]:
from rul_pm.dataset.CMAPSS import CMAPSSDataset, sensor_indices
from rul_pm.transformation.transformers import Transformer, transformation_pipeline
from rul_pm.models.keras import  KerasTrainableModel, XiangQiangJianQiaoModel
from rul_pm.models.keras.losses import weighted_categorical_crossentropy
from rul_pm.models.keras.attention import Attention
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from rul_pm.iterators.batcher import get_batcher, dataset_map, get_features
from tensorflow.keras import backend as K, Model, Input, optimizers

In [2]:
from tensorflow.keras.layers import TimeDistributed 
from tensorflow.keras.layers import (Layer, Conv1D, Dense, Add, Input, AveragePooling1D,
                                     BatchNormalization, LayerNormalization, Flatten,
                                     Concatenate, Bidirectional, Conv2D, MaxPooling1D,
                                     Permute, TimeDistributed, Multiply, LeakyReLU,    
                                     GlobalAveragePooling1D, MaxPooling2D,
                                     Softmax, AveragePooling2D,
                                     ConvLSTM2D ,SpatialDropout2D, LocallyConnected1D,
                                     multiply, concatenate, Activation, Masking,
                                     Masking, LSTM, GRU, MaxPool1D, Conv1D, Dropout, Average,
                                     Reshape, UpSampling1D, AveragePooling1D,GaussianNoise,
                                     Concatenate, Bidirectional)
from tcn import TCN
from tensorflow.keras import backend as K, Model, Input, optimizers
import tensorflow as tf

In [3]:
train_dataset = CMAPSSDataset(train=True, model='FD001')
test_dataset = CMAPSSDataset(train=False, model='FD001')

FD001


In [91]:
sensors = ['SensorMeasure1',
       'SensorMeasure2', 'SensorMeasure3', 'SensorMeasure4', 'SensorMeasure5',
       'SensorMeasure6', 'SensorMeasure7', 'SensorMeasure8', 'SensorMeasure9',
       'SensorMeasure10', 'SensorMeasure11', 'SensorMeasure12',
       'SensorMeasure13', 'SensorMeasure14', 'SensorMeasure15',
       'SensorMeasure16', 'SensorMeasure17', 'SensorMeasure18',
       'SensorMeasure19', 'SensorMeasure20', 'SensorMeasure21']

s1 = [sensors[i-1] for i in sensor_indices]

In [11]:
class RawAndBinClasses(BaseEstimator, TransformerMixin):   
    """
        A target transformer that outputs
        the RUL + nbins binary vectors
    """
    def __init__(self, nbins):
        self.nbins = nbins 
        
        
    def fit(self, X, y=None):       
        self.max_RUL = int(X.max())
        self.value_ranges = np.linspace(0, self.max_RUL, num=self.nbins+1)        
        return self
    
    def transform(self, X):
        v = X        
        classes = []
        for j in range(len(self.value_ranges)-1):                 
            lower = self.value_ranges[j] 
            upper = self.value_ranges[j+1] 
            classes.append(((v>=lower) & (v <upper)))        
        v = np.vstack((v, *classes)).T
        return v

class SoftmaxRegression(KerasTrainableModel):
    """
    The network contains stacked layers of 1-dimensional convolutional layers
    followed by max poolings

    Parameters
    ----------
    self: list of tuples (filters: int, kernel_size: int)
          Each element of the list is a layer of the network. The first element of the tuple contaings
          the number of filters, the second one, the kernel size.
    """

    def __init__(self, raw_and_bins, alpha, window, batch_size, step, transformer, shuffle, models_path,
                 patience=4, cache_size=30, output_size=3, padding='same'):
        super(SoftmaxRegression, self).__init__(window,
                                                  batch_size,
                                                  step,
                                                  transformer,
                                                  shuffle,
                                                  models_path,
                                                  patience=patience,
                                                  output_size=output_size,
                                                  cache_size=30,
                                                  callbacks=[])
        if raw_and_bins is not None:
            self.raw_and_bins = raw_and_bins
            weights = [1 for _ in range(self.raw_and_bins.nbins)]
            self.wcc = weighted_categorical_crossentropy(weights)
            self.output_size = self.raw_and_bins.nbins
        else:
            self.output_size = 1
        self.alpha = alpha
        
    
    def _generate_batcher(self, train_batcher, val_batcher):
        n_features = self.transformer.n_features
        def gen_train():
            for X, y in train_batcher:
                yield X, y

        def gen_val():
            for X, y in val_batcher:
                yield X, y

        a = tf.data.Dataset.from_generator(
            gen_train, (tf.float32, tf.float32), 
            (tf.TensorShape([None, self.window, n_features]), 
             tf.TensorShape([None, self.output_size])))
        b = tf.data.Dataset.from_generator(
            gen_val, (tf.float32, tf.float32), 
            (tf.TensorShape([None, self.window, n_features]), 
             tf.TensorShape([None, self.output_size])))
        return a,b
    
    def _loss(self, y_true, y_pred):
        # cross entropy loss
        bin_true = y_true[:,1:]
        cont_true = y_true[:,0]        
        
        #y_pred_rul = y_pred[:, 0]
        #y_pred_bins = y_pred[:, 1:]
        y_pred_bins = y_pred
        cls_loss = self.wcc(bin_true, y_pred_bins)
        # MSE loss
        idx_tensor = self.raw_and_bins.value_ranges[:-1]
        pred_cont = tf.reduce_sum(y_pred_bins * idx_tensor, 1)
        #pred_cont = tf.keras.backend.argmax(y_pred, axis=1)
        rmse_loss_softmax = tf.losses.mean_squared_error(cont_true, pred_cont)
        
        #mse_loss = tf.losses.mean_squared_error(cont_true, y_pred_rul)
        # Total loss
        total_loss = (cls_loss +
                      self.alpha * rmse_loss_softmax
                     )
        return total_loss
    

    def mse_softmax(self, y_true, y_pred):
         # cross entropy loss
        bin_true = y_true[:,1:]
        cont_true = y_true[:,0]        
        
        # y_pred_rul = y_pred[:, 0]
        # y_pred_bins = y_pred[:, 1:]
        y_pred_bins = y_pred
        idx_tensor = self.raw_and_bins.value_ranges[:-1]
        pred_cont = tf.reduce_sum(y_pred_bins * idx_tensor, 1)
        #pred_cont = tf.keras.backend.argmax(y_pred, axis=1)
        return tf.sqrt(tf.losses.mean_squared_error(cont_true, pred_cont))
    

    def mse_rul(self,  y_true, y_pred):
         # cross entropy loss
        cont_true = y_true[:,0]        
        y_pred_rul = y_pred[:, 0]
        return tf.losses.mean_squared_error(cont_true, y_pred_rul)     
        
    def compile(self):
        self.compiled=  True
        self.model.compile(loss='mse', 

                           optimizer=tf.keras.optimizers.Adam(lr=0.0001))
        


    def build_model(self):
        from collections import namedtuple
        #function to split the input in multiple outputs
        def splitter(x):
            return [x[:,:,i:i+1] for i in range(n_features)]

        n_features = self.transformer.n_features
        
        i = Input(shape=(self.window, n_features))   
        


        m = TCN(use_skip_connections=True, 
                use_batch_norm=True, 
                return_sequences=True,
                dropout_rate=0.1)(i)    
        m = Attention(64, self.window-1)(m)
        m = Dense(100, activation='relu')(m)                
        m = Dropout(0.5)(m)        
        m = BatchNormalization()(m)
        proba = Dense(150, activation='relu')(m)        
        proba = BatchNormalization()(proba)
        proba = Dropout(0.1)(proba)
        proba = Dense(1, activation='linear')(proba)
        

        return Model(inputs = i, outputs =proba)



    @property
    def name(self):
        return 'ConvolutionalSimple'

    def get_params(self, deep=False):
        params = super().get_params()
        params.update({
        })
        return params


# 1. Remaining useful life estimation in prognostics using deep convolution neural networks
Author 
* Xiang Li
* Qian Ding
* Jian-Qiao Sunc

https://www.sciencedirect.com/science/article/pii/S0951832017307779?casa_token=DHKuBa83HrcAAAAA:-U6kHeTYyqmo9XYB8Wm-hOFOx-2IMOC_o5bhZEpdEW8tTB8zliBx9kzxiFuqX6pu_lf7nAQDqeq-#tbl0002


A deep learning method for prognostics is proposed based on convolution neural networks. Dropout technique is employed to relieve overfitting problem. Experiments are carried out on the popular C-MAPSS dataset to show the effectiveness of the proposed method. The goal of the task is to estimate the remaining useful life of aero-engine units accurately. With raw feature selection, data pre-processing and sample preparation using time window, good prognostic performance is achieved with the proposed method, and small error between the prediction and the actual RUL value is obtained for the testing data. The RUL in the life-time of the engine units can be well predicted, especially for the late period close to failure.


In [68]:
transformer = Transformer(
                'RUL',
                transformation_pipeline(                                
                    scaler=MinMaxScaler((-1, 1)),
                    features=sensors
                )
            )



In [143]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=0)) 
    
def scheduler(epoch, lr): 
    if epoch < 200:
        return 0.001
    else:
        return 0.0001
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
model = XiangQiangJianQiaoModel(
            10,
            10,           
            dropout=0.5,
            window=30,
            batch_size=32,
            step = 1,
            transformer=transformer,
            shuffle='all',
            models_path=Path('.'),
            patience=500,
            cache_size=40,
            callbacks=[lr_callback],
            learning_rate=1e-3,
            loss=root_mean_squared_error)

In [144]:
def get_matrix(df):
    w = 30
    X = []
    y = []
    for life in df['life'].unique():
        df_life = df[df['life'] == life].reset_index()
        for i in range(0, df_life.shape[0]-w):
            X.append(np.expand_dims(df_life.iloc[i:i+w, :][s1].values, 0))
            y.append(np.expand_dims(df_life.iloc[i+w, :]['RUL'], 0))
    return np.concatenate(X), np.concatenate(y)

scaler = MinMaxScaler((-1, 1))
train_df = train_dataset.toPandas()
test_df = test_dataset.toPandas()

train_df[sensors] = X_train = scaler.fit_transform(train_df[sensors])

test_df[sensors] = X_train = scaler.transform(test_df[sensors])

X_train, y_train = get_matrix(train_df)
X_test, y_test = get_matrix(test_df)




In [165]:
from rul_pm.models.keras.layers import ExpandDimension
from tensorflow.keras.layers import (GRU, LSTM, RNN, Activation, Add,
                                     AveragePooling1D, BatchNormalization,
                                     Bidirectional, Concatenate, Conv1D,
                                     Conv2D, Dense, Dropout, Flatten,
                                     GaussianNoise, Lambda, Layer,
                                     LayerNormalization, LSTMCell, Masking,
                                     MaxPool1D, Permute, Reshape,
                                     SpatialDropout1D, StackedRNNCells,
                                     UpSampling1D, ZeroPadding2D)
import math 

n_filters = 10
filter_size = 10
dropout = 0.5
n_features = len(s1)
input = Input(shape=(30, n_features))
x = input

x = ExpandDimension()(x)
x = Conv2D(n_filters, (filter_size, 1),
           padding='same', activation='tanh',
           )(x)
x = Conv2D(n_filters, (filter_size, 1),
           padding='same', activation='tanh',
           )(x)
x = Conv2D(n_filters, (filter_size, 1),
           padding='same', activation='tanh',
           )(x)
x = Conv2D(n_filters, (filter_size, 1),
           padding='same', activation='tanh')(x)
x = Conv2D(1, (3,1), padding='same', activation='tanh')(x)

x = Flatten()(x)
x = Dropout(dropout)(x)
x = Dense(100,
          activation='tanh')(x)
output = Dense(1)(x)
mm = Model(
    inputs=[input],
    outputs=[output],
)

In [169]:
np.any(np.isnan(X_train))

False

In [166]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 
mm.summary()
mm.compile(
            loss=root_mean_squared_error,
            optimizer=optimizers.Adam(lr=0.001, beta_1=0.85, beta_2=0.9, epsilon=0.001, amsgrad=True))
mm.fit(X_train, X_train,
       validation_data=(X_test, y_test),
       batch_size=512,
       shuffle=True,
       epochs=200)

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 30, 14)]          0         
_________________________________________________________________
lambda_8 (Lambda)            (None, 30, 14, 1)         0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 30, 14, 10)        110       
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 30, 14, 10)        1010      
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 30, 14, 10)        1010      
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 30, 14, 10)        1010      
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 30, 14, 1)         31  

35/35 [==============================] - 5s 148ms/step - loss: 23.8077 - val_loss: 32.5094
Epoch 63/200
35/35 [==============================] - 5s 147ms/step - loss: 23.4604 - val_loss: 33.6729
Epoch 64/200
35/35 [==============================] - 5s 147ms/step - loss: 23.6203 - val_loss: 33.3799
Epoch 65/200
35/35 [==============================] - 5s 147ms/step - loss: 23.3703 - val_loss: 31.8527
Epoch 66/200
35/35 [==============================] - 5s 149ms/step - loss: 23.2162 - val_loss: 31.8167
Epoch 67/200
35/35 [==============================] - 5s 147ms/step - loss: 23.1489 - val_loss: 31.5043
Epoch 68/200
35/35 [==============================] - 6s 158ms/step - loss: 22.9681 - val_loss: 31.3419
Epoch 69/200
35/35 [==============================] - 5s 152ms/step - loss: 22.9359 - val_loss: 31.5225
Epoch 70/200
35/35 [==============================] - 5s 148ms/step - loss: 22.9551 - val_loss: 31.7245
Epoch 71/200
35/35 [==============================] - 5s 148ms/step - loss: 2

KeyboardInterrupt: 

In [ ]:
model.fit(train_dataset, test_dataset, epochs=500)

In [ ]:

v = model.predict(train_dataset, step=5)
#predicted_probas = v
#idx_tensor = transformer.transformerY['scale'].value_ranges[:-1]
#pred_cont = np.sum(predicted_probas * idx_tensor, axis=1)
pred_cont = v
true_data = get_features(train_dataset, step=5, window=30, features=['RUL'])
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
ax.plot(true_data['RUL'][0:530], label='True')
ax.plot(pred_cont[0:500], label='Predicted')
ax.legend()

In [ ]:

v = model.predict(test_dataset, step=1)
#predicted_probas = v
#idx_tensor = transformer.transformerY['scale'].value_ranges[:-1]
#pred_cont = np.sum(predicted_probas * idx_tensor, axis=1)
pred_cont = v
true_data = get_features(test_dataset, step=1, window=30, features=['RUL'])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 5))
ax.plot(true_data['RUL'][0:1000], label='True')
ax.plot(pred_cont[0:1000], label='Predicted')
ax.legend()

In [ ]:
np.concatenate(model.true_values(test_dataset, step=5)).shape

In [ ]:
model.true_values(test_dataset, step=5)

In [ ]:
test_dataset[0]['RUL']

###### 